# Data

## Data Sources
- [x] Home and rent prices - Zillow: https://www.zillow.com/research/data/
- [x] Household Income - ACS: https://www.census.gov/programs-surveys/acs
- [x] Unemployment - ACS: https://www.census.gov/programs-surveys/acs
- [x] Distance to Disney World - Google Maps API: https://developers.google.com/maps/documentation

# Data Download

In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
from census import Census
from us import states

c:\Users\brad\miniconda3\envs\ds2025-group3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Zillow
Zillow provides a dataset of home prices and rent prices for various regions in the US. The data is available in CSV format and can be downloaded from their website. The dataset includes information on home and rent prices by neighborhood for a variety of large cities across the US.

In [3]:
# Read the Zillow CSV file into a pandas DataFrame
zillow = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zhvi/Neighborhood_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1742438344")

# Filter only the data for the city of Orlando, FL
zillow = zillow[zillow['City'] == 'Orlando']

# Drop redundant and unnecessary columns
zillow = zillow.drop(columns=['RegionID', 'RegionType', 'State', 'StateName', 'SizeRank', 'City', 'Metro', 'CountyName', 'SizeRank'])

# Convert zillow into long format
zillow = zillow.melt(id_vars='RegionName', var_name='period', value_name='value')

# Find median home value for each neighborhood annually
zillow['period'] = pd.to_datetime(zillow['period']).dt.year
zillow = zillow.groupby(['RegionName', 'period']).median().reset_index()
zillow = zillow.rename(columns={'RegionName': 'neighborhood'})

## American Community Survey (ACS)
The ACS provides a wealth of demographic and economic data, including household income and unemployment rates. The data is available in CSV format and can be downloaded from the Census Bureau's website. The dataset includes information on household income and unemployment rates by neighborhood for a variety of large cities across the US.

In [8]:
c = Census(os.getenv("CENSUS_API"))

# Household Income
income = []
for year in range(2009, 2022):
  hhi = c.acs5.state_county_tract(('NAME', 'B19013_001E'),
                         states.FL.fips,
                         county_fips='095',
                         tract='*',
                         year=year)
  i_df = pd.DataFrame(hhi)
  i_df['tract'] = i_df['tract'].astype(int)
  i_df['value'] = i_df['B19013_001E']
  i_df = i_df[['tract', 'value']]
  i_df['tract'] = i_df['tract'].astype(str).str.zfill(6)
  i_df['period'] = year
  income.append(i_df)

income = pd.concat(income, ignore_index=True)

# Unemployment Rate
unemployment = []
for year in range(2011, 2023):
  unem = c.acs5.state_county_tract(('NAME', 'B23025_003E', 'B23025_005E'),
                         states.FL.fips,
                         county_fips='095',
                         tract='*',
                         year=year)
  u_df = pd.DataFrame(unem)
  u_df['tract'] = u_df['tract'].astype(int)
  u_df['value'] = u_df['B23025_005E'] / u_df['B23025_003E']
  u_df = u_df[['tract', 'value']]
  u_df['tract'] = u_df['tract'].astype(str).str.zfill(6)
  u_df['period'] = year
  unemployment.append(u_df)

unemployment = pd.concat(unemployment, ignore_index=True)

## Distance to Disney World
The distance to Disney World can be calculated using the Google Maps API. The API provides a way to calculate the distance between two locations using their latitude and longitude coordinates. The distance can be calculated in miles or kilometers, depending on the user's preference. The API also provides information on travel time and directions, which can be useful for users who are planning a trip to Disney World.

# Geographic Downloads
The target geographic scale is neighborhoods, but Census publishes tract data that does not match exactly with neighborhoods.

In [9]:
import geopandas as gpd
import requests
from zipfile import ZipFile
import io

## Neighborhoods

In [10]:
# Load a GeoJSON file containing the geographical boundaries of Orlando neighborhoods
url = "https://raw.githubusercontent.com/blackmad/neighborhoods/refs/heads/master/orlando.geojson"
orlando_geo = gpd.read_file(url)

## Census Tracts

In [11]:
workspace_path = os.getenv("WORKSPACE_PATH")

url = "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_12_tract_500k.zip"

response = requests.get(url)
with ZipFile(io.BytesIO(response.content)) as z:
  z.extractall(workspace_path + 'data/')

tracts = gpd.read_file(workspace_path + 'data/cb_2018_12_tract_500k.shp')
tracts = tracts[tracts['COUNTYFP'] == '095']

### Mapping Census Tracts to Neighborhoods

In [12]:
# Map each tract to its corresponding neighborhood
tracts['GEOID'] = tracts['GEOID'].astype(str).str.zfill(11)
tracts['neighborhood'] = None
for index, row in tracts.iterrows():
    tract_geom = row['geometry']
    for _, neighborhood_row in orlando_geo.iterrows():
        if tract_geom.intersects(neighborhood_row['geometry']):
            tracts.at[index, 'neighborhood'] = neighborhood_row['name']
            break
tracts = tracts[tracts['neighborhood'].notnull()]
tracts = tracts[['GEOID', 'neighborhood']]
tracts['GEOID'] = tracts['GEOID'].str[5:]
tracts['GEOID'] = tracts['GEOID'].astype(str).str.zfill(6)
tracts = tracts.rename(columns={'GEOID': 'tract', 'neighborhood': 'RegionName'})

# Database
SQL commands can be used to create a database that stores the data from the Zillow and ACS datasets. The database can be created using a variety of database management systems, including MySQL, PostgreSQL, and SQLite. The SQL commands can be used to create tables for each dataset, as well as to insert the data into the tables. The database can then be queried to retrieve information on home prices, rent prices, household income, and unemployment rates by neighborhood.

In [13]:
import sqlite3

## Create Tables and Schema

In [14]:
with open('ddl.txt', 'r') as sql_file:
  sql_script = sql_file.read()

with sqlite3.connect('orlando.db') as conn:
  conn.executescript(sql_script)

## Add Initial Data

In [15]:
with sqlite3.connect('orlando.db') as conn:
  zillow.to_sql('zillow', conn, if_exists='append', index=False)
  income.to_sql('income', conn, if_exists='append', index=False)
  unemployment.to_sql('unemployment', conn, if_exists='append', index=False)
  tracts.to_sql('tracts', conn, if_exists='append', index=False)